In [13]:
import scipy.special
import scipy.misc
import matplotlib.pyplot as plt
import numpy as np
import datetime
from sklearn.externals import joblib
class neuralNetwork:
    # 初始化神经网络
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        # 初始化输入层、隐藏层、输出层的节点数
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes

        # 初始化输入层-隐藏层权重、隐藏层-输出层权重
        # 下图函数生成高斯分布的概率密度随机矩阵，数学经验法则为：采用节点传入链接数量的平方根的倒数，pow()函数用于幂计算，最后一个参数为希望输出的随机数形状
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))

        # 设置学习率
        self.lr = learningrate

        # sigmod函数计算
        self.activation_function = lambda x: scipy.special.expit(x)  # 使用lambda关键词创建匿名函数，返回scipy.special.expit(x) 其中expit()函数代表

    # 训练神经网络
    def train(self,inputs_list,targets_list):
        #将输入值以及目标值换为二维矩阵
        inputs = np.array(inputs_list,ndmin=2).T
        targets = np.array(targets_list,ndmin=2).T

        #计算到隐藏层的信号输入值
        hidden_inputs = np.dot(self.wih,inputs)
        # 计算到隐藏层的信号输出值
        hidden_outputs = self.activation_function(hidden_inputs)
        # 计算到末尾输出层的信号输入值
        final_inputs = np.dot(self.who,hidden_outputs)
        # 计算末尾输出层的信号输出值
        final_outputs = self.activation_function(final_inputs)

        output_errors = targets - final_outputs #计算输出层的误差
        hidden_errors = np.dot(self.who.T,output_errors) #计算隐藏层的误差

        #更新隐藏层和输出层之间的链接权重
        self.who += self.lr * np.dot((output_errors*final_outputs*(1 - final_outputs)),np.transpose(hidden_outputs))
        #更新输入层和隐藏层之间的链接权重
        self.wih += self.lr * np.dot((hidden_errors*hidden_outputs*(1 - hidden_outputs)),np.transpose(inputs))
    # 查询神经网络
    def query(self, inputs_list):
        # convert inputs list into 2d array
        inputs = np.array(inputs_list, ndmin=2).T

        # 计算到隐藏层的输入信号
        hidden_inputs = np.dot(self.wih, inputs)
        # 计算隐藏层的输出值
        hidden_outputs = self.activation_function(hidden_inputs)

        # 计算到输出层的输入讯号
        final_inputs = np.dot(self.who, hidden_outputs)
        # 计算输出层的输出值
        final_outputs = self.activation_function(final_inputs)
        return final_outputs
    def getWih(self):
        return self.wih
    def getWho(self):
        return self.who

In [14]:
#初始化神经网络对象
n = neuralNetwork(784,200,10,0.1) #4个参数分别对应输入节点 隐藏层节点数目 输出层节点数目 学习率

#积分卡 用于紧跟着的第二段准确度测试程序
scorecard = []

In [15]:
#加载训练数据
training_data_file = open("mnist_dataset/mnist_train.csv",'r')
training_data_list = training_data_file.readlines()
training_data_file.close()

In [16]:
#预处理训练数据
print("Training is begining! Time:",datetime.datetime.now())
for i in range(5):
    for record in training_data_list:
        all_values = record.split(',') #将原始数据以‘，’为标志切割开    
        inputs = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01  #将原始数据进行缩放至 0.01 ~ 1.0 范围内（由于0值会使权重更新失败）
        #初始化目标输出（由于sigmod函数的存在 神经网络的输出值在0.0~1.0之间 为了正确的分配预测标签 将十个数字标签分别分配给10个输出节点）
        targets = np.zeros(10) + 0.01  # 生成一个大小为10 的数组 所有的值均为 0.01
        targets[int(all_values[0])] = 0.99 #将原始数据的第一个值all_values[0] 也就是数据标签 的目标值初始化为 0.99  这样整个目标值矩阵除了正确目标的目标值为 0.99 其余的均为0.01
        n.train(inputs,targets)
        pass
    print(i/5*100,"%")
    pass
print("Training is finnished! Time:",datetime.datetime.now())

Training is begining! Time: 2019-07-29 17:38:45.538882
0.0 %
20.0 %
40.0 %
60.0 %
80.0 %
Training is finnished! Time: 2019-07-29 18:08:31.752378


In [17]:
#加载测试数据
test_data_file = open("mnist_dataset/mnist_test.csv", 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

In [22]:
#测试网络
for record  in test_data_list:
    #预处理测试数据
    all_values = record.split(',')
    #将正确值（标签）取出来
    correct_label = int(all_values[0]) 
    
    #查询神经网络
    inputs = (np.asfarray(all_values[1:])/255.0 * 0.99) + 0.01#定义查询输入
    outputs = n.query(inputs)
    prodict_label = np.argmax(outputs)#将十个输出节点中 值最大的索引（索引刚好是对应的数字）
    
    if(prodict_label == correct_label):
        scorecard.append(1)   #如果标签值和预测值一致 加一分
    else:
        scorecard.append(0)   #如果标签值和预测值一致 不计分
        pass
    pass
#转化为准确比
scorecard_array = np.asarray(scorecard)
print("准确率为：",scorecard_array.sum() / scorecard_array.size *100,"%")
save_wih = n.getWih()
save_who = n.getWho()
joblib.dump(save_wih,"H:/Jupyter Notebook/mnist_dataset/Model_weight/mnist_model_wih.pkl")  #将模型训练好之后的参数保存到本地
joblib.dump(save_who,"H:/Jupyter Notebook/mnist_dataset/Model_weight/mnist_model_who.pkl")  #将模型训练好之后的参数保存到本地

准确率为： 97.18 %


['H:/Jupyter Notebook/mnist_dataset/Model_weight/mnist_model_who.pkl']